### Get info for arrivals to BER Airport

##### Using AeroDataBox API from RapidAPI

In [61]:
from datetime import datetime, timedelta, timezone
from IPython.display import display, JSON
import pandas as pd
import requests

In [99]:
# Get tomorrow's date

tomorrow_time_date = datetime.now() + timedelta(days=1)
tomorrow = tomorrow_time_date.strftime('%Y-%m-%d')

In [102]:
# For Between 00:00 - 12:00 o'clock

API_key = "****************************************"

airport_icoa = "EDDB"
tomorrow = tomorrow
to_local_time = f"{tomorrow}T00:00"
from_local_time = f"{tomorrow}T12:00"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
	"X-RapidAPI-Key": API_key
}

response_am = requests.request("GET", url, headers=headers, params=querystring)

response_json_am = response_am.json()
display(JSON(response_json_am))

<IPython.core.display.JSON object>

In [103]:
# Between 12:00 - 23:59 o'clock

API_key = "****************************************"

airport_icoa = "EDDB"
to_local_time = f"{tomorrow}T12:00"
from_local_time = f"{tomorrow}T23:59"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
	"X-RapidAPI-Key": API_key
}

response_pm = requests.request("GET", url, headers=headers, params=querystring)

response_json_pm = response_pm.json()
display(JSON(response_json_pm))

<IPython.core.display.JSON object>

In [64]:
# Transform the data from json format to a pd data frame

arrivals_first_half = pd.json_normalize(response_am.json()["arrivals"])
arrivals_second_half = pd.json_normalize(response_pm.json()["arrivals"])

In [77]:
# arrivals_first_half.info()

In [78]:
# arrivals_second_half.info()

In [67]:
# select the columns you want to include into the database for the am flights

arrivals_first_half = (
    arrivals_first_half
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
    'arrival.terminal','status','aircraft.model'])
    .assign(icao_arrival_airport = airport_icoa)
)


In [79]:
# arrivals_first_half.info()

In [69]:
# select the columns you want to include into the database for the pm flights

arrivals_second_half = (
    arrivals_second_half
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
    'arrival.terminal','status','aircraft.model'])
    .assign(icao_arrival_airport = airport_icoa)
)

In [80]:
# arrivals_second_half.info()

In [71]:
# Append the half day data frames into one df which covers the whole day

df_ber_flights_arrivals = arrivals_first_half.append(arrivals_second_half)

In [89]:
# df_ber_flights_arrivals.info()

In [87]:
# df_ber_flights_arrivals.tail(10)

In [91]:
# Replace all null values with "unknown"

import numpy as np
df_ber_flights_arrivals = (
    df_ber_flights_arrivals
    .replace({np.nan},'unknown'))
    #.assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    

# change date datatype
df_ber_flights_arrivals["arrival.scheduledTimeLocal"] = pd.to_datetime(df_ber_flights_arrivals["arrival.scheduledTimeLocal"])

In [93]:
# df_ber_flights_arrivals.info()

### Update data into database using sqlalchemy

#### Establish the connection

In [94]:
import sqlalchemy

In [95]:
schema="name of the schema"
host="provide the aws host"
user="admin"
password="your password for MySQL"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Import and update the tables to the SQL

In [96]:
df_ber_flights_arrivals.to_sql('arrivals', con=con, if_exists='append', index=False)